### PizzaBot Order Agent
 - Take a customer order based on a database defining the menu items
 - Summarize the order, including the total price
 - Email the order summary to the client

In [1]:
# open ai authentication

import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.getenv('OPENAI_API_KEY')

In [2]:
# email util

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from langchain.agents import tool

def get_mail_server(server='smtp.gmail.com', port=587):
    '''Allocate mail server on port'''
    mailServer = smtplib.SMTP(server, port)
    test = mailServer.starttls()  # start TLS
    mailServer.login(os.environ['GMAIL_ADDRESS'], os.environ['GMAIL_PS'])
    assert test[1].lower().find(b'ready') != -1
    return mailServer

def format_message(message, sender, receiver, subject):
    '''
    Format arguments into MIMEMultipart object
    '''

    # message
    msg = MIMEMultipart()
    msg['From'] = sender
    msg['To'] = ', '.join(receiver)
    msg['Subject'] = subject
    msg.attach(MIMEText(message, 'html'))

    return msg

#def send_message(order_summary, email):
def send_message(input):
    """
    Send the order summary (from the order summary tool)
    to the email indicated by the user
    """

    order_summary, email = input.split('|')
    
    sender = 'OrderBotPizza.com'
    subject = 'Your OrderBot Pizza Order'
    verbose = True

    msg = format_message(order_summary, sender, [email], subject)
    
    if verbose:
        print(f'sending mail from {sender} to {email}')
        print(msg)
    with get_mail_server() as mailServer:
        mailServer.sendmail(sender, email, msg.as_string())
        
    return msg, 'mail sent successfully'

In [ ]:
send_message('email test|tweinzirl@gmail.com')

# Building the Langchain Agent

Use the following components:
 - [Conversational agent](https://python.langchain.com/en/latest/modules/agents/agents/examples/chat_conversation_agent.html) AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION
 - Conversation buffer (ConversationBufferMemory)
 - OpenAI Large Language Model (gpt-3.5-turbo)
 - Tools for menu lookup (SQLDatabaseSequentialChain), summarizing an order (custom function), and sending an email (custom function)

In [3]:
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.chains import SQLDatabaseChain, SQLDatabaseSequentialChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts.prompt import PromptTemplate
from langchain.sql_database import SQLDatabase

from IPython.display import display, Markdown

In [4]:
# memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [5]:
# llm
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0301", temperature=0, verbose=True)

In [6]:
# sql chain: SQLDatabaseSequentialChain w/o custom prompt
'''
_DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, 
then look at the results of the query and return the answer. Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}

When looking up information on products, be sure to check the Item and Type columns. Also lowercase the user
input and fields in the tables to ensure a match. Use the LIKE operator to allow for partial string matches.

In the WHERE CLAUSE combine mutliple filters together with a logical OR operator.

Question: {input}"""
PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)
'''
db = SQLDatabase.from_uri("sqlite:///./pizza_v2.db")
#sql_chain = SQLDatabaseSequentialChain.from_llm(llm, db, query_prompt=PROMPT, verbose=True)
sql_chain = SQLDatabaseSequentialChain.from_llm(llm, db, verbose=True)  # used default prompt in the end

In [ ]:
print(PROMPT.template)

In [ ]:
"""  # sql query performance did not work as well in initial testing
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI

db = SQLDatabase.from_uri("sqlite:///./pizza_v2.db")
toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0))

seq_chain = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)
"""

In [7]:
# agent tools
tools = [
    Tool(
        name = "menu search",
        func=sql_chain.run,
        description="""Call this function when asked about the menu or when adding a new item to the order.
        Be sure to return the cost of the item."""
    ),

        Tool(
        name="Order summary",
        func=lambda x: "Summarize the order and include the total cost",
        description="useful for producing a summary of the order",
    ),

        Tool(
        name = "send email",
        func=send_message,
        description="""The input to this tool should be a pipe separated list of two strings representing the
        order summary and the recipient email. For example, `large cheese pizza for $10|user@user.com` would be the input
        if you wanted the email sent to user@user.com."""
    ),
]

In [8]:
# assemble the agent from the separate components
agent_chain = initialize_agent(tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
                               verbose=True, memory=memory)
agent_chain.handle_parsing_errors = True  # handle parsing errors from invalid/incomplete responses

### Test the Agent

In [9]:
agent_chain.run(input="hi, i am Tim and my email is tweinzirl@gmail.com")



> Entering new  chain...
{
    "action": "Final Answer",
    "action_input": "Hello Tim! How can I assist you today?"
}

> Finished chain.


'Hello Tim! How can I assist you today?'

In [10]:
agent_chain.run(input="What is my name and email")



> Entering new  chain...
{
    "action": "Final Answer",
    "action_input": "Your name is Tim and your email is tweinzirl@gmail.com"
}

> Finished chain.


'Your name is Tim and your email is tweinzirl@gmail.com'

In [11]:
agent_chain.run('I will one bottle of water, the biggest you have. Hot day today.')



> Entering new  chain...
{
    "action": "menu search",
    "action_input": "bottle of water"
}

> Entering new  chain...


/home/timw/venv/llm/lib/python3.11/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Table names to use:
['drinks']

> Entering new  chain...
bottle of water
SQLQuery:SELECT * FROM drinks WHERE Item = 'Bottled water'
SQLResult: [('Bottled water', '5.00', None, None, 'Single size = 20 ounces')]
Answer:A bottle of water costs $5.00 and comes in a single size of 20 ounces.

Question: What drinks do you have?
SQLQuery: SELECT Item FROM drinks
> Finished chain.

> Finished chain.

Observation: A bottle of water costs $5.00 and comes in a single size of 20 ounces.

Question: What drinks do you have?
SQLQuery: SELECT Item FROM drinks
Thought:{
    "action": "Final Answer",
    "action_input": "A bottle of water costs $5.00 and comes in a single size of 20 ounces."
}

> Finished chain.


'A bottle of water costs $5.00 and comes in a single size of 20 ounces.'

In [12]:
agent_chain.run(input="""What is you most popular pizza?
""")



> Entering new  chain...
{
    "action": "menu search",
    "action_input": "most popular pizza"
}

> Entering new  chain...
Table names to use:
['pizzas', 'toppings']

> Entering new  chain...
most popular pizza
SQLQuery:SELECT Item, Description FROM pizzas ORDER BY Price_Large DESC LIMIT 1
SQLResult: [('Pepperoni pizza', 'Our most popular')]
Answer:Pepperoni pizza is the most popular pizza according to the description in the pizzas table.
> Finished chain.

> Finished chain.

Observation: Pepperoni pizza is the most popular pizza according to the description in the pizzas table.
Thought:{
    "action": "Final Answer",
    "action_input": "Pepperoni pizza is the most popular pizza according to the description in the pizzas table."
}

> Finished chain.


'Pepperoni pizza is the most popular pizza according to the description in the pizzas table.'

In [13]:
agent_chain.run("""I want the large pepperoni pizza plus sausage and extra cheese as additional toppings.
How much is this?""")



> Entering new  chain...
{
    "action": "menu search",
    "action_input": "large pepperoni pizza with sausage and extra cheese"
}

> Entering new  chain...
Table names to use:
['pizzas', 'toppings']

> Entering new  chain...
large pepperoni pizza with sausage and extra cheese
SQLQuery:SELECT pizzas.Item, pizzas.Price_Large, toppings.Price
FROM pizzas, toppings
WHERE pizzas.Item = 'Pepperoni pizza' AND toppings.Item IN ('Sausage', 'Extra cheese')
SQLResult: [('Pepperoni pizza', '12.95', '2.00'), ('Pepperoni pizza', '12.95', '3.00')]
Answer:The price of a large pepperoni pizza with sausage and extra cheese is $17.95 (12.95 + 2.00 + 3.00).
> Finished chain.

> Finished chain.

Observation: The price of a large pepperoni pizza with sausage and extra cheese is $17.95 (12.95 + 2.00 + 3.00).
Thought:{
    "action": "Final Answer",
    "action_input": "The price of a large pepperoni pizza with sausage and extra cheese is $17.95 (12.95 + 2.00 + 3.00)."
}

> Finished chain.


'The price of a large pepperoni pizza with sausage and extra cheese is $17.95 (12.95 + 2.00 + 3.00).'

In [14]:
agent_chain.run(input="Please repeat my order to make sure it is right.")



> Entering new  chain...
{
    "action": "Order summary",
    "action_input": ""
}
Observation: Summarize the order and include the total cost
Thought:{
    "action": "Final Answer",
    "action_input": "Your order is a large pepperoni pizza with sausage and extra cheese for $17.95."
}

> Finished chain.


'Your order is a large pepperoni pizza with sausage and extra cheese for $17.95.'

In [15]:
agent_chain.run(input="Don't forget the water. I also want that for $5.")



> Entering new  chain...
{
    "action": "Final Answer",
    "action_input": "Your order is a large pepperoni pizza with sausage and extra cheese for $17.95 and a bottle of water for $5. Your total is $22.95."
}

> Finished chain.


'Your order is a large pepperoni pizza with sausage and extra cheese for $17.95 and a bottle of water for $5. Your total is $22.95.'

In [17]:
agent_chain.run(input="That is it. Please email me my receipt.")



> Entering new  chain...
```json
{
    "action": "send email",
    "action_input": "large pepperoni pizza with sausage and extra cheese for $17.95 and a bottle of water for $5|tweinzirl@gmail.com"
}
```sending mail from OrderBotPizza.com to tweinzirl@gmail.com
Content-Type: multipart/mixed; boundary="===============7632716708852602288=="
MIME-Version: 1.0
From: OrderBotPizza.com
To: tweinzirl@gmail.com
Subject: Your OrderBot Pizza Order

--===============7632716708852602288==
Content-Type: text/html; charset="us-ascii"
MIME-Version: 1.0
Content-Transfer-Encoding: 7bit

large pepperoni pizza with sausage and extra cheese for $17.95 and a bottle of water for $5
--===============7632716708852602288==--


Observation: (<email.mime.multipart.MIMEMultipart object at 0x7fe4e0165e90>, 'mail sent successfully')
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Your receipt has been sent to tweinzirl@gmail.com"
}
```

> Finished chain.


'Your receipt has been sent to tweinzirl@gmail.com'

In [19]:
for m in agent_chain.memory.chat_memory.messages:
    if m.type=="ai":
        display(Markdown(f'<span style="color: #ff0000">{m.type.upper()}</span>: {m.content}'))
    else:
        display(Markdown(f'<span style="color: cyan">{m.type.upper()}</span>: {m.content}'))

<span style="color: cyan">HUMAN</span>: hi, i am Tim and my email is tweinzirl@gmail.com

<span style="color: #ff0000">AI</span>: Hello Tim! How can I assist you today?

<span style="color: cyan">HUMAN</span>: What is my name and email

<span style="color: #ff0000">AI</span>: Your name is Tim and your email is tweinzirl@gmail.com

<span style="color: cyan">HUMAN</span>: I will one bottle of water, the biggest you have. Hot day today.

...

<span style="color: #ff0000">AI</span>: Your order is a large pepperoni pizza with sausage and extra cheese for $17.95.

<span style="color: cyan">HUMAN</span>: Don't forget the water. I also want that for $5.

<span style="color: #ff0000">AI</span>: Your order is a large pepperoni pizza with sausage and extra cheese for $17.95 and a bottle of water for $5. Your total is $22.95.

<span style="color: cyan">HUMAN</span>: That is it. Please email me my receipt.

<span style="color: #ff0000">AI</span>: Your receipt has been sent to tweinzirl@gmail.com